In [2]:
%cd ..

/home/romet/projects/ut/e2e-rally-estonia


In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm.auto import tqdm

import torch
import torch.nn as nn
import torchvision

from dataloading.nvidia import NvidiaDataset, NvidiaTrainDataset, NvidiaValidationDataset, Normalize
from pilotnet import PilotnetControl, PilotNetConditional, PilotNetConditionalOld
from metrics.metrics import calculate_open_loop_metrics
from viz.analytics import create_waypoint_error_plot

from metrics.metrics import calculate_total_distance, calculate_closed_loop_metrics, calculate_trajectory_open_loop_metrics, calculate_open_loop_metrics, read_frames_driving, read_frames_expert
from trajectory import calculate_steering_angle
import math

from trainer import ControlTrainer, ConditionalTrainer
from torchvision import transforms

from camera_frame import CameraFrameTransformer


%load_ext autoreload 
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
root_path = Path("/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021")
summer_dev_dataset = [
            root_path / "2021-05-20-12-36-10_e2e_sulaoja_20_30",
            root_path / "2021-05-20-12-43-17_e2e_sulaoja_20_30",
            root_path / "2021-05-20-12-51-29_e2e_sulaoja_20_30",
            root_path / "2021-05-20-13-44-06_e2e_sulaoja_10_10",
            root_path / "2021-05-20-13-51-21_e2e_sulaoja_10_10",
            root_path / "2021-05-20-13-59-00_e2e_sulaoja_10_10",
            root_path / "2021-05-28-15-07-56_e2e_sulaoja_20_30",
            root_path / "2021-05-28-15-17-19_e2e_sulaoja_20_30",
            {'path': root_path / "2021-06-09-13-14-51_e2e_rec_ss2", 'start': 125, 'end': 49725},
            {'path': root_path / "2021-06-09-13-55-03_e2e_rec_ss2_backwards", 'start': 150, 'end': 53625},
            {'path': root_path / "2021-06-09-14-58-11_e2e_rec_ss3", 'start': 175, 'end': 43775},
            {'path': root_path / "2021-06-09-15-42-05_e2e_rec_ss3_backwards", 'start': 100, 'end': 40625},
            root_path / "2021-06-09-16-24-59_e2e_rec_ss13",
            root_path / "2021-06-09-16-50-22_e2e_rec_ss13_backwards",
            root_path / "2021-06-10-12-59-59_e2e_ss4",
            root_path / "2021-06-10-13-19-22_e2e_ss4_backwards",
            root_path / "2021-06-10-13-51-34_e2e_ss12",
            root_path / "2021-06-10-14-02-24_e2e_ss12_backwards",
            root_path / "2021-06-10-14-44-24_e2e_ss3_backwards",
            root_path / "2021-06-10-15-03-16_e2e_ss3_backwards",
            root_path / "2021-06-14-11-08-19_e2e_rec_ss14",
            root_path / "2021-06-14-11-22-05_e2e_rec_ss14",
            root_path / "2021-06-14-11-43-48_e2e_rec_ss14_backwards",
            {'path': root_path / "2021-09-24-11-19-25_e2e_rec_ss10", 'start': 400, 'end': 34550},
            {'path': root_path / "2021-09-24-11-40-24_e2e_rec_ss10_2", 'start': 150, 'end': 16000},
            {'path': root_path / "2021-09-24-12-02-32_e2e_rec_ss10_3", 'start': 350, 'end': 8050},
            root_path / "2021-09-24-12-21-20_e2e_rec_ss10_backwards",
            root_path / "2021-09-24-13-39-38_e2e_rec_ss11",
            {'path': root_path / "2021-09-30-13-57-00_e2e_rec_ss14", 'start': 100, 'end': 3200},
            root_path / "2021-09-30-15-03-37_e2e_ss14_from_half_way",
            root_path / "2021-09-30-15-20-14_e2e_ss14_backwards",
            {'path': root_path / "2021-09-30-15-56-59_e2e_ss14_attempt_2", 'start': 80, 'end': 54600},
            root_path / "2021-10-07-11-05-13_e2e_rec_ss3",
            root_path / "2021-10-07-11-44-52_e2e_rec_ss3_backwards",
            root_path / "2021-10-07-12-54-17_e2e_rec_ss4",
            root_path / "2021-10-07-13-22-35_e2e_rec_ss4_backwards",
            root_path / "2021-10-11-16-06-44_e2e_rec_ss2",
            root_path / "2021-10-11-17-10-23_e2e_rec_last_part",
            root_path / "2021-10-11-17-14-40_e2e_rec_backwards",
            root_path / "2021-10-11-17-20-12_e2e_rec_backwards",
            root_path / "2021-10-20-14-55-47_e2e_rec_vastse_ss13_17",
            root_path / "2021-10-20-13-57-51_e2e_rec_neeruti_ss19_22",
            root_path / "2021-10-20-14-15-07_e2e_rec_neeruti_ss19_22_back",
            root_path / "2021-10-25-17-31-48_e2e_rec_ss2_arula",
            root_path / "2021-10-25-17-06-34_e2e_rec_ss2_arula_back",
        ]

summer_val_dataset = [
            root_path / "2021-05-28-15-19-48_e2e_sulaoja_20_30",
            root_path / "2021-06-07-14-20-07_e2e_rec_ss6",
            root_path / "2021-06-07-14-06-31_e2e_rec_ss6",
            root_path / "2021-06-07-14-09-18_e2e_rec_ss6",
            root_path / "2021-06-07-14-36-16_e2e_rec_ss6",
            root_path / "2021-09-24-14-03-45_e2e_rec_ss11_backwards",
            root_path / "2021-10-26-10-49-06_e2e_rec_ss20_elva",
            root_path / "2021-10-26-11-08-59_e2e_rec_ss20_elva_back",
            root_path / "2021-10-20-15-11-29_e2e_rec_vastse_ss13_17_back",
            {'path': root_path / "2021-10-11-14-50-59_e2e_rec_vahi", 'start': 100, 'end': 15000},
            {'path': root_path / "2021-10-14-13-08-51_e2e_rec_vahi_backwards", 'start': 80, 'end': 13420},
            root_path / "2022-06-10-13-23-01_e2e_elva_forward",
            root_path / "2022-06-10-13-03-20_e2e_elva_backward"
        ]


root_path = Path("/home/romet/data2/datasets/rally-estonia/winter2022")

winter_dev_dataset = [
            root_path / '2021-11-08-11-24-44_e2e_rec_ss12_raanitsa',
            root_path / '2021-11-08-12-08-40_e2e_rec_ss12_raanitsa_backward',
            root_path / "2022-01-28-10-21-14_e2e_rec_peipsiaare_forward",
            root_path / "2022-01-28-12-46-59_e2e_rec_peipsiaare_backward",
            root_path / "2022-01-14-10-05-16_e2e_rec_raanitsa_forward",
            root_path / "2022-01-14-10-50-05_e2e_rec_raanitsa_backward",
            root_path / "2022-01-14-11-54-33_e2e_rec_kambja_forward2",
            root_path / "2022-01-14-12-21-40_e2e_rec_kambja_forward2_continue",
            root_path / "2022-01-14-13-09-05_e2e_rec_kambja_backward",
            root_path / "2022-01-14-13-18-36_e2e_rec_kambja_backward_continue",
            root_path / "2022-01-14-12-35-13_e2e_rec_neeruti_forward",
            root_path / "2022-01-14-12-45-51_e2e_rec_neeruti_backward",
            root_path / "2022-01-18-13-03-03_e2e_rec_arula_backward",
            root_path / "2022-01-18-13-43-33_e2e_rec_otepaa_forward",
            root_path / "2022-01-18-13-52-35_e2e_rec_otepaa_forward",
            root_path / "2022-01-18-13-56-22_e2e_rec_otepaa_forward",
            root_path / "2022-01-18-14-12-14_e2e_rec_otepaa_backward",
            root_path / "2022-01-18-15-20-35_e2e_rec_kanepi_forward",
            root_path / "2022-01-18-15-49-26_e2e_rec_kanepi_backwards",
        ]
    
winter_val_dataset = [
            root_path / "2022-01-18-12-37-01_e2e_rec_arula_forward",
            root_path / "2022-01-18-12-47-32_e2e_rec_arula_forward_continue",
            root_path / "2022-01-28-14-47-23_e2e_rec_elva_forward",
            root_path / "2022-01-28-15-09-01_e2e_rec_elva_backward",
            root_path / "2022-01-25-15-25-15_e2e_rec_vahi_forward",
            root_path / "2022-01-25-15-34-01_e2e_rec_vahi_backwards",
        ]


In [27]:

def calculate_stats(dataset):
    total_frames = 0
    total_distance = 0
    for dataset_path in dataset:
        if type(dataset_path) is dict:
            dataset_path = dataset_path['path']
        frames_df = pd.read_csv(dataset_path / "nvidia_frames.csv")
        #print(f"{dataset_path} {len(frames_df)}")
        total_frames += len(frames_df)
        total_distance += calculate_total_distance(frames_df)
    return {
        "total_frames": total_frames,
        "total_distance": total_distance
    }

In [28]:
print("Summer dev: ", calculate_stats(summer_dev_dataset))
print("Summer val: ", calculate_stats(summer_val_dataset))
print("Winter dev: ", calculate_stats(winter_dev_dataset))
print("Winter val: ", calculate_stats(winter_val_dataset))

Summer dev:  {'total_frames': 1321033, 'total_distance': 428219.6515249652}
Summer dev:  {'total_frames': 288735, 'total_distance': 101391.4746362972}
Winter dev:  {'total_frames': 792149, 'total_distance': 276811.06173890614}
Winter val:  {'total_frames': 131561, 'total_distance': 46198.05446120254}


In [29]:
1321033 + 288735

1609768

In [33]:
288735 / 1321033

0.21856759066578957

In [30]:
428219.6515249652 + 101391.4746362972

529611.1261612624

In [31]:
792149 + 131561

923710

In [32]:
276811.06173890614 + 46198.05446120254

323009.11620010866

In [34]:
tr = transforms.Compose([Normalize()])
dataset_path = Path("/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021")
summer_cleaned_dev = NvidiaTrainDataset(dataset_path, n_branches=3, metadata_file="nvidia_frames_ext.csv")

augmentation: color_prob=0.0, noise_prob=0.0, blur_prob=0.0
/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-05-28-15-07-56_e2e_sulaoja_20_30: lenght=15623, filtered=3
/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-05-28-15-17-19_e2e_sulaoja_20_30: lenght=3215, filtered=3
/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-06-09-13-14-51_e2e_rec_ss2: lenght=49589, filtered=330
/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-06-09-13-55-03_e2e_rec_ss2_backwards: lenght=53452, filtered=291
/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-06-09-14-58-11_e2e_rec_ss3: lenght=42888, filtered=998
/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-06-09-15-42-05_e2e_rec_ss3_backwards: lenght=40524, filtered=170
/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-06-09-16-24-59_e2e_rec_ss13: lenght=18043, filtered=560
/h

In [35]:
tr = transforms.Compose([Normalize()])
dataset_path = Path("/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021")
summer_cleaned_val = NvidiaValidationDataset(dataset_path, n_branches=3, metadata_file="nvidia_frames_ext.csv")

/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-05-28-15-19-48_e2e_sulaoja_20_30: lenght=10705, filtered=3
/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-06-07-14-20-07_e2e_rec_ss6: lenght=25833, filtered=4
/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-06-07-14-06-31_e2e_rec_ss6: lenght=3002, filtered=1
/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-06-07-14-09-18_e2e_rec_ss6: lenght=4550, filtered=2
/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-06-07-14-36-16_e2e_rec_ss6: lenght=25351, filtered=18
/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-09-24-14-03-45_e2e_rec_ss11_backwards: lenght=25167, filtered=5
/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-10-26-10-49-06_e2e_rec_ss20_elva: lenght=33034, filtered=11
/home/romet/data2/datasets/rally-estonia/dataset-new-small/summer2021/2021-10-26-

In [36]:
summer_cleaned_val.frames

,level_0,index,front_wide_filename,autonomous,left_filename,right_filename,steering_angle,cmd_steering_angle,vehicle_speed,turn_signal,...,wp10_left_z,wp10_right_x,wp10_right_y,wp10_right_z,steering_angle_left,steering_angle_right,row_id,yaw_delta,image_path,camera_type
0,0,2021-05-28 12:19:49.042497873,front_wide/1622204389042497873.png,False,left/1622204389042497873.png,right/1622204389042497873.png,-0.156194,NaN,10.343286,1,...,-1.702256,47.712896,-3.428181,-3.261214,-8.143722,6.088260,0,-0.001042,/home/romet/data2/datasets/rally-estonia/datas...,front_wide
1,1,2021-05-28 12:19:49.075848818,front_wide/1622204389075848818.png,False,left/1622204389075848818.png,right/1622204389075848818.png,-0.166755,NaN,10.357756,1,...,-1.739623,47.770248,-3.385567,-3.300913,-8.194614,6.120693,1,-0.001221,/home/romet/data2/datasets/rally-estonia/datas...,front_wide
2,2,2021-05-28 12:19:49.109150887,front_wide/1622204389109150887.png,False,left/1622204389109150887.png,right/1622204389109150887.png,-0.176509,NaN,10.360000,1,...,-1.755415,47.832043,-3.330673,-3.319290,-9.019704,6.983181,2,-0.001041,/home/romet/data2/datasets/rally-estonia/datas...,front_wide
3,3,2021-05-28 12:19:49.142489910,front_wide/1622204389142489910.png,False,left/1622204389142489910.png,right/1622204389142489910.png,-0.188435,NaN,10.413987,1,...,-1.794883,47.887260,-3.284164,-3.361057,-8.966115,6.898134,3,-0.001320,/home/romet/data2/datasets/rally-estonia/datas...,front_wide
4,4,2021-05-28 12:19:49.175845861,front_wide/1622204389175845861.png,False,left/1622204389175845861.png,right/1622204389175845861.png,-0.203846,NaN,10.330598,1,...,-1.816290,47.543172,-3.218255,-3.371690,-8.958357,6.872480,4,-0.001099,/home/romet/data2/datasets/rally-estonia/datas...,front_wide
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32721,32725,2022-06-10 10:21:32.265486002,front_wide/1654856492265486002.png,False,left/1654856492265486002.png,right/1654856492265486002.png,0.253483,NaN,9.065191,1,...,NaN,NaN,NaN,NaN,NaN,NaN,32725,0.000884,/home/romet/data2/datasets/rally-estonia/datas...,front_wide
32722,32726,2022-06-10 10:21:32.298826933,front_wide/1654856492298826933.png,False,left/1654856492298826933.png,right/1654856492298826933.png,0.244512,NaN,9.067340,1,...,NaN,NaN,NaN,NaN,NaN,NaN,32726,0.001014,/home/romet/data2/datasets/rally-estonia/datas...,front_wide
32723,32727,2022-06-10 10:21:32.332163811,front_wide/1654856492332163811.png,False,left/1654856492332163811.png,right/1654856492332163811.png,0.240423,NaN,9.032628,1,...,NaN,NaN,NaN,NaN,NaN,NaN,32727,0.001144,/home/romet/data2/datasets/rally-estonia/datas...,front_wide
32724,32728,2022-06-10 10:21:32.365483999,front_wide/1654856492365483999.png,False,left/1654856492365483999.png,right/1654856492365483999.png,0.234038,NaN,9.042611,1,...,NaN,NaN,NaN,NaN,NaN,NaN,32728,0.000908,/home/romet/data2/datasets/rally-estonia/datas...,front_wide


In [39]:
summer_dev_total_frames = len(summer_cleaned_dev.frames)
print(f"Summer clean dev: {summer_dev_total_frames} frames")

summer_val_total_frames = len(summer_cleaned_val.frames)
print(f"Summer clean dev: {summer_val_total_frames} frames")

Summer clean dev: 1093804 frames
Summer clean dev: 276409 frames


In [40]:
1093804 + 276409

1370213

In [41]:
(1609768-1370213)/1609768

0.1488133693799355